In [1]:
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

%matplotlib inline
plt.style.use("fivethirtyeight")
sns.set_context("notebook")

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = {}
with open("../examples/data/birth_life_2010.txt", "r") as file:
    for line in file.readlines()[1:]:
        temp = line.split("\t")
        data[temp[0]] = (float(temp[1]), float(temp[2][:-1]))
        
data = pd.DataFrame(data).transpose()
data.columns = ["birth_rate", "life_expectancy"]
data.head()

,birth_rate,life_expectancy
Afghanistan,6.288,48.282195
Albania,1.536,76.900951
Algeria,2.264,72.852537
Angola,5.443,50.653659
Argentina,2.211,75.632146


In [3]:
# Convert dataframe into features and targets nd-arrays
X_input = data["birth_rate"].values
target = data["life_expectancy"].values

# Shuffle the data
np.random.seed(123)
perms = np.random.permutation(X_input.shape[0])
X_input = X_input[perms]
target = target[perms]

X_input.shape, target.shape

((190,), (190,))

# Linear Regression using Placeholders

In [7]:
g = tf.Graph()

with g.as_default():
    X = tf.placeholder(dtype=tf.float32, shape=None, name="X")
    y = tf.placeholder(dtype=tf.float32, shape=None, name="y")
    
    w = tf.get_variable(name="weight", initializer=tf.constant(0, tf.float32))
    b = tf.get_variable(name="bias", initializer=tf.constant(0, tf.float32))
    
    y_pred = w * X + b
    
    loss = tf.square(y - y_pred)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.09).minimize(loss)
    
    init = tf.global_variables_initializer()

placeholder_time = []

for i in range(100):
    start = time.time()
    with tf.Session(graph=g) as sess:
        sess.run(init)

        for epoch in range(10):
            for x_, y_ in zip(X_input, target):
                sess.run([optimizer], feed_dict={X: x_, y: y_})
    end = time.time()
    placeholder_time.append(end - start)
    print(f"Time taken: {end - start:.2f}seconds")   

Time taken: 0.38seconds
Time taken: 0.39seconds
Time taken: 0.39seconds
Time taken: 0.38seconds
Time taken: 0.39seconds
Time taken: 0.37seconds
Time taken: 0.38seconds
Time taken: 0.39seconds
Time taken: 0.38seconds
Time taken: 0.38seconds
Time taken: 0.45seconds
Time taken: 0.38seconds
Time taken: 0.39seconds
Time taken: 0.38seconds
Time taken: 0.38seconds
Time taken: 0.38seconds
Time taken: 0.38seconds
Time taken: 0.38seconds
Time taken: 0.38seconds
Time taken: 0.39seconds
Time taken: 0.42seconds
Time taken: 0.40seconds
Time taken: 0.40seconds
Time taken: 0.39seconds
Time taken: 0.39seconds
Time taken: 0.39seconds
Time taken: 0.39seconds
Time taken: 0.37seconds
Time taken: 0.37seconds
Time taken: 0.37seconds
Time taken: 0.38seconds
Time taken: 0.37seconds
Time taken: 0.37seconds
Time taken: 0.40seconds
Time taken: 0.37seconds
Time taken: 0.38seconds
Time taken: 0.37seconds
Time taken: 0.37seconds
Time taken: 0.39seconds
Time taken: 0.38seconds
Time taken: 0.37seconds
Time taken: 0.63

# Linear Regression using Dataset

In [8]:
g = tf.Graph()

with g.as_default():
    train_data = tf.data.Dataset.from_tensor_slices((X_input, target))
    iterator = train_data.make_initializable_iterator()
    x, y = iterator.get_next()
    
    w = tf.get_variable(name="weight", initializer=tf.constant(0, tf.float64))
    b = tf.get_variable(name="bias", initializer=tf.constant(0, tf.float64))
    
    y_pred = w * x + b
    
    loss = tf.square(y - y_pred)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.09).minimize(loss)
    
    init = tf.global_variables_initializer()

dataset_time = []

for i in range(100):
    start = time.time()
    with tf.Session(graph=g) as sess:
        sess.run(init)

        for epoch in range(10):
            sess.run(iterator.initializer)
            try:
                while True:
                    sess.run([optimizer])
            except tf.errors.OutOfRangeError:
                pass
            
    end = time.time()
    dataset_time.append(end - start)
    print(f"Time taken: {end - start:.2f}seconds")    

Time taken: 0.27seconds
Time taken: 0.27seconds
Time taken: 0.28seconds
Time taken: 0.28seconds
Time taken: 0.27seconds
Time taken: 0.27seconds
Time taken: 0.27seconds
Time taken: 0.29seconds
Time taken: 0.28seconds
Time taken: 0.28seconds
Time taken: 0.27seconds
Time taken: 0.28seconds
Time taken: 0.27seconds
Time taken: 0.30seconds
Time taken: 0.28seconds
Time taken: 0.30seconds
Time taken: 0.27seconds
Time taken: 0.40seconds
Time taken: 0.37seconds
Time taken: 0.33seconds
Time taken: 0.28seconds
Time taken: 0.33seconds
Time taken: 0.28seconds
Time taken: 0.27seconds
Time taken: 0.29seconds
Time taken: 0.27seconds
Time taken: 0.29seconds
Time taken: 0.28seconds
Time taken: 0.31seconds
Time taken: 0.30seconds
Time taken: 0.27seconds
Time taken: 0.40seconds
Time taken: 0.42seconds
Time taken: 0.39seconds
Time taken: 0.34seconds
Time taken: 0.33seconds
Time taken: 0.30seconds
Time taken: 0.29seconds
Time taken: 0.26seconds
Time taken: 0.26seconds
Time taken: 0.26seconds
Time taken: 0.26

In [9]:
np.mean(dataset_time) / np.mean(placeholder_time)

0.664419676983342